In [49]:
import sys
from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

import pandas as pd
import numpy as np

In [50]:
TIMEOUT = 5

driver = None

In [56]:
df = pd.read_csv('kakaoGoogle.csv', encoding='utf-8-sig')

len(df[df['review_address'] == '리뷰 없음'])

115

In [52]:
def search_review(driver, df, index, p_name):
  place = []
  user = []
  rating = []
  comment = []
  enter = False
  try:
    info_list = driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd').find_elements(By.CSS_SELECTOR, 'div.Nv2PK.THOPZb.CpccDe')
    
    if len(info_list) == 0:
      info_list = driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd').find_elements(By.CSS_SELECTOR, 'div.Nv2PK.tH5CWc.THOPZb')
    
    for info in info_list:
      if df.loc[index, 'title'] in info.find_element(By.CSS_SELECTOR, 'div.qBF1Pd.fontHeadlineSmall').text:
        info.find_element(By.XPATH, '..').click()
        enter = True
        break
      else:
        continue
    if enter == True:
      try:
        driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span[2]/span[1]/span').click()
        elem = driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf')
        
        review_count = driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf > div.PPCwl > div > div.jANrlb > div.fontBodySmall').text.replace('리뷰 ', '').replace('개', '')
        
        if ',' in review_count:
          review_count = review_count.replace(',', '')
          
        review_count = int(review_count)
          
        if review_count >= 200:
          review_count = 200
        
        for i in range(review_count//5):
          driver.execute_script("arguments[0].scrollBy(0, 5000)", elem)
          sleep(0.5)
      
        review_boxes = driver.find_elements(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf > div:nth-child(9) > div')
        
        if len(review_boxes) <review_count:
          review_boxes = driver.find_elements(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf > div:nth-child(8) > div')
          
        if review_count<200:
          for i in range(0, (review_count)*3, 3):
            print('p_name: ', p_name)
            place.append(p_name)
            
            user_name = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.WNxzHc.qLhwHc > a > div.d4r55 > span').text
            print('user_name: ', user_name)
            user.append(user_name)
          
            place_rating = int(review_boxes[i].find_element(By.CSS_SELECTOR, 'span.kvMYJc').get_attribute('aria-label').replace('별표 ', '').replace('개', ''))
            # place_rating = int(review_boxes[i].find_element(By.CSS_SELECTOR, 'span.fzvQIb').text.replace('/5', ''))
            print('place_rating: ', place_rating)
            rating.append(place_rating)
          
            # place_comment = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.GHT2ce > span.wiI7pd').text
            place_comment = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.MyEned > span.wiI7pd').text
            print('place_comment: ', place_comment)
            comment.append(place_comment)
            
          review_df = pd.DataFrame({
              'place_id':place,
              'user_id':user,
              'score':rating,
              'comment':comment
          },)
          
          try:
            if df.loc[index, 'review_address'] != '리뷰 없음':
              old_review_df = pd.read_csv(df.loc[index, 'review_address'], encoding='utf-8-sig')
              review_df = pd.concat([review_df, old_review_df])
          except:
            pass

          review_df.to_csv(f'./review/{p_name}.csv', index = False, header=True, encoding='utf-8-sig')
          return f'./review/{p_name}.csv'
          
        else:
          for i in range(0, 200*3, 3):
            print('p_name: ', p_name)
            place.append(p_name)
            
            user_name = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.WNxzHc.qLhwHc > a > div.d4r55 > span').text
            print('user_name: ', user_name)
            user.append(user_name)
          
            place_rating = int(review_boxes[i].find_element(By.CSS_SELECTOR, 'span.kvMYJc').get_attribute('aria-label').replace('별표 ', '').replace('개', ''))
            # place_rating = int(review_boxes[i].find_element(By.CSS_SELECTOR, 'span.fzvQIb').text.replace('/5', ''))
            print('place_rating: ', place_rating)
            rating.append(place_rating)
          
            # place_comment = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.GHT2ce > span.wiI7pd').text
            place_comment = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.MyEned > span.wiI7pd').text
            print('place_comment: ', place_comment)
            comment.append(place_comment)
            
          review_df = pd.DataFrame({
              'place_id':place,
              'user_id':user,
              'score':rating,
              'comment':comment
          },)
          
          try:
            if df.loc[index, 'review_address'] != '리뷰 없음':
              old_review_df = pd.read_csv(df.loc[index, 'review_address'], encoding='utf-8-sig')
              review_df = pd.concat([review_df, old_review_df])
          except:
            pass

          review_df.to_csv(f'./review/{p_name}.csv', index = False, header=True, encoding='utf-8-sig')
          return f'./review/{p_name}.csv'
          
      except Exception as e:
        print(e)
        return '리뷰 없음'

  except Exception as e:
    try:
      driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span[2]/span[1]/span').click()
      elem = driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf')
      
      review_count = driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf > div.PPCwl > div > div.jANrlb > div.fontBodySmall').text.replace('리뷰 ', '').replace('개', '')
      
      if ',' in review_count:
        review_count = review_count.replace(',', '')
        
      review_count = int(review_count)
          
      if review_count >= 200:
        review_count = 200
      
      for i in range(review_count//5):
        driver.execute_script("arguments[0].scrollBy(0, 5000)", elem)
        sleep(0.5)
      
      review_boxes = driver.find_elements(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf > div:nth-child(8) > div')
      
      if len(review_boxes) < review_count:
        review_boxes = driver.find_elements(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf > div:nth-child(9) > div')
      
      print(len(review_boxes))
      
      if review_count<200:
        for i in range(0, (review_count)*3, 3):
          print('p_name: ', p_name)
          place.append(p_name)
            
          user_name = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.WNxzHc.qLhwHc > a > div.d4r55 > span').text
          print('user_name: ', user_name)
          user.append(user_name)
          
          place_rating = int(review_boxes[i].find_element(By.CSS_SELECTOR, 'span.kvMYJc').get_attribute('aria-label').replace('별표 ', '').replace('개', ''))
          # place_rating = int(review_boxes[i].find_element(By.CSS_SELECTOR, 'span.fzvQIb').text.replace('/5', ''))
          print('place_rating: ', place_rating)
          rating.append(place_rating)
          
          # place_comment = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.GHT2ce > span.wiI7pd').text
          place_comment = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.MyEned > span.wiI7pd').text
          print('place_comment: ', place_comment)
          comment.append(place_comment)
            
        review_df = pd.DataFrame({
            'place_id':place,
            'user_id':user,
            'score':rating,
            'comment':comment
        },)
        
        try:
          if df.loc[index, 'review_address'] != '리뷰 없음':
            old_review_df = pd.read_csv(df.loc[index, 'review_address'], encoding='utf-8-sig')
            review_df = pd.concat([review_df, old_review_df])
        except:
          pass

        review_df.to_csv(f'./review/{p_name}.csv', index = False, header=True, encoding='utf-8-sig')
        return f'./review/{p_name}.csv'
          
      else:
        for i in range(0, 200*3, 3):
          print('p_name: ', p_name)
          place.append(p_name)
            
          user_name = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.WNxzHc.qLhwHc > a > div.d4r55 > span').text
          print('user_name: ', user_name)
          user.append(user_name)
          
          place_rating = int(review_boxes[i].find_element(By.CSS_SELECTOR, 'span.kvMYJc').get_attribute('aria-label').replace('별표 ', '').replace('개', ''))
          # place_rating = int(review_boxes[i].find_element(By.CSS_SELECTOR, 'span.fzvQIb').text.replace('/5', ''))
          print('place_rating: ', place_rating)
          rating.append(place_rating)
          
          # place_comment = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.GHT2ce > span.wiI7pd').text
          place_comment = review_boxes[i].find_element(By.CSS_SELECTOR, 'div.MyEned > span.wiI7pd').text
          print('place_comment: ', place_comment)
          comment.append(place_comment)
            
        review_df = pd.DataFrame({
            'place_id':place,
            'user_id':user,
            'score':rating,
            'comment':comment
        },)
        
        try:
          if df.loc[index, 'review_address'] != '리뷰 없음':
            old_review_df = pd.read_csv(df.loc[index, 'review_address'], encoding='utf-8-sig')
            review_df = pd.concat([review_df, old_review_df])
        except:
          pass

        review_df.to_csv(f'./review/{p_name}.csv', index = False, header=True, encoding='utf-8-sig')
        return f'./review/{p_name}.csv'
          
    except Exception as e:
      print(e)
      return '리뷰 없음'

In [53]:
for search in range(495):
  Options = webdriver.ChromeOptions()
  user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'

  Options.add_argument('user-agent=' + user_agent)

  driver = webdriver.Chrome("./chromedriver", options=Options)

  driver.get("https://www.google.co.kr/maps")
  driver.implicitly_wait(TIMEOUT)
  elem = driver.find_element(By.ID, "searchboxinput")
  # elem.send_keys(df.loc[search_index[i], 'p_category']+' '+df.loc[search_index[i], 'p_name'])
  elem.send_keys('울산 '+df.loc[search, 'title'])
  # 검색 단추를 누른다
  elem = driver.find_element(By.ID, "searchbox-searchbutton")
  elem.click()
  name = df.loc[search, 'title']
  review_address = search_review(driver, df, search, name)
  df.loc[search, 'review_address'] = review_address
  
  driver.quit()

C:\Users\82107\AppData\Local\Temp\ipykernel_32036\270387098.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./chromedriver", options=Options)


809
p_name:  일산해수욕장
user_name:  Mitchell Kim
place_rating:  4
place_comment:  평일이라그런지 한산한 해변. 날씨가 그다지 덥지않아서인지 해수욕하는사람은 없었습니다. 공용화장실 깨끗하고 휴지도 비치되어있으며 관리가 잘 되고있습니다.샤워시설도 잘 운영되고있으며 백사장 길이는 짧지만 폭이 매우 넓어서 광활한 느낌이 듭니다.
p_name:  일산해수욕장
user_name:  HyunJi Jang
place_rating:  5
place_comment:  바닷물에 제대로 들어가 본 적은 없는 곳이지만 봄여름가을겨울 사계절 언제가도 좋은 곳이에요. 가게들이 많은데도 거리가 잘 정돈되어있고, 이번엔 마켓이 열려서 구경도 하고 좋았네요. 가게들도 관광지치고 바가지 씌우지 않아서 좋아요.
p_name:  일산해수욕장
user_name:  허혜정
place_rating:  4
place_comment:  도심에 관광지스러운 해수욕장이 있다니!!
저는 타지역에 살다가 울산으로 이사왔는데
마치 부산 해운대 미니 버전같은 느낌이었어요…
p_name:  일산해수욕장
user_name:  j자스민
place_rating:  5
place_comment:  한가하고 우리만에 바다가 된듯 여유롭고 몰랐는데요. 현대라한호텔투숙객에게는 무료로  수건.썬베드,샤워쿠폰까지 서비스해서 너무좋았어요.수영하기전에 고소공포증있는 저도 달달 떨면서 건너봤는데요.다리건너고 오세요.
p_name:  일산해수욕장
user_name:  풀꽃향기
place_rating:  3
place_comment:  울산일산해수욕장
반달형 모래사장 해변이 예쁘네요^^
해변 주변 카페나 식당등이 많아서 좋아요!…
p_name:  일산해수욕장
user_name:  Pome Dongdong
place_rating:  5
place_comment:  생각보다 큰 일산해수욕장..
근처에 대왕암도 있어서 놀러가기 좋구요..
최근엔 출렁다리도 생겨서 관광객들이 많이 보입니다..…

C:\Users\82107\AppData\Local\Temp\ipykernel_32036\270387098.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./chromedriver", options=Options)


509
p_name:  태화강국가정원
user_name:  Hong Ju Seo
place_rating:  5
place_comment:  맑고 깨끗하게 정돈된 느낌 좋네요. 근데 주변에 까마귀는 왜 이리 많아요?  까마귀와 돗거하는 주변 마을 분들 대단합니다. 마치 까마구가 없다는듯 편안하네요. 외지에서 처음 온 분들은 깜짝 놀랄수도 있겠네요!
p_name:  태화강국가정원
user_name:  Bart Chang
place_rating:  5
place_comment:  정원...집안에 있는 뜰이나 꽃밭

우리 나라를 대표할 만큼 충분한 정원으로 인정합니다…
p_name:  태화강국가정원
user_name:  hyun jo park
place_rating:  4
place_comment:  십리대숲길이 있는 국가정원으로
대왕암공원과 함께 울산시내의
대표적인 관광명소…
p_name:  태화강국가정원
user_name:  Daehyun Kim
place_rating:  5
place_comment:  언젠가 울산 방문을 하면 꼭 방문해야할 곳중 한곳이었는데
작년에 울산을 들렀는데 못들러서 올해 무리해서 간곳
내 노력이 헛되지 않았다.
너무나 아름다운 국가정원이었다.
p_name:  태화강국가정원
user_name:  안효정
place_rating:  5
place_comment:  봄에는 꽃구경하러 가기 좋지만 사계절에 가도 그 계절마다 아름다운공원입니다~.
애들 킥보드나 자전거 들고 가서 한바퀴하거나 텐트 치고 놀기에 좋아요~^^
p_name:  태화강국가정원
user_name:  에코쉬림프
place_rating:  4
place_comment:  양귀비 등 대형 풀꽃단지가 어디인지 궁금했는데
지도에서 태화강 북쪽 강변에서 주차하시고
만남의광장에서 내려가면 가까워요.…
p_name:  태화강국가정원
user_name:  최찬돌 (찬돌이)
place_rating:  4
place_comment:  주변에 산책로가 많아 산책하기 좋네요. 인근에 식당도 많이 있

C:\Users\82107\AppData\Local\Temp\ipykernel_32036\270387098.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./chromedriver", options=Options)


...

287
p_name:  팜핑레이크관광농원
user_name:  DONG IN Kim
place_rating:  4
place_comment:  저수지 뷰와 야경뷰가 좋고 샤워시설 매우 좋습니다. 아이들이 좋아하는 방방이도 있고요. 수영장 시설도 좋지만 수영장 수온이 조금 차가운 편이라서 아쉬웠네요.
p_name:  팜핑레이크관광농원
user_name:  뉴깜
place_rating:  4
place_comment:  주차장 넓음 . 조금걸어야함.
수영장 수질좋고 넓고 깨끗함.물은 차가운편.
주위에 산책로랑 카페.작은동물 있음.…
p_name:  팜핑레이크관광농원
user_name:  다라온TV
place_rating:  4
place_comment:  샤워실에 찬물이 나오고 오후에 여자 화장실에 냄새빼고는 좋았어요~
물에 약품 냄새가 났지만 뭐 수영장이니 어쩔수없죠~미끄럼틀 하나 없지만 아이들이 너무 즐거워해서 또 가고싶어요~
p_name:  팜핑레이크관광농원
user_name:  박마도로스
place_rating:  5
place_comment:  아기데리고가기좋은곳,수영장은 낮은곳과깊은곳으로놔누어져있음.
p_name:  팜핑레이크관광농원
user_name:  이사람
place_rating:  5
place_comment:  글램핑 텐트 컨디션이 별로 안좋습니다

침구류 따로 챙기셔야 합니다…
p_name:  팜핑레이크관광농원
user_name:  KR JANG
place_rating:  5
place_comment:  마근저수지위에 멋진캠핑장 생겼네요~
아이들 물놀이하기 좋겠고 각종식물 등 멋진곳입니다^^
p_name:  팜핑레이크관광농원
user_name:  Emma Kim
place_rating:  5
place_comment:  오토캠핑 2회했어요.
조경 시설 다 좋아요
단, 사람이 몰리면 샤워실 따뜻한물 안 나옴 ㅠㅠ
p_name:  팜핑레이크관광농원
user_name:  부산타이어파크
place_rating:  5
place_comment:  너무

C:\Users\82107\AppData\Local\Temp\ipykernel_32036\270387098.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./chromedriver", options=Options)


p_name:  청량천
user_name:  Alex Kim
place_rating:  5
place_comment:  회야강 하류에 조성해 놓은 청량천의 산책로다.
조성한지 일년도 되지않았다. 깔끔한 산책로와 자전거도로가 잘 정비되어 있다.
산책로 길가에는 파크골프장이 운영되고 있고 화단,가로수들이 즐비하게 늘어서 있다.…
p_name:  청량천
user_name:  하하블리
place_rating:  5
place_comment:  
20
p_name:  다운동고분군
user_name:  생활다락방
place_rating:  1
place_comment:  여기가 왜 고분군인지 막상 보면 모르겠습니다. 흔적도 없이 밭만 무성한 그런 곳인데...
p_name:  다운동고분군
user_name:  세은
place_rating:  3
place_comment:  저희집 근처인데...하.하.하 정말 그냥 산 인줄알았어요. 하하
가끔 새소리,부엉이소리? 나서 학교갈때 좋아요 근데...누가 사는지는 모르지만... 담배냄시가...하..어떤 태우는 냄시가..
가끔 고통스러워요...하하..하
p_name:  다운동고분군
user_name:  김재화
place_rating:  1
place_comment:  관리가 되지않아서 보기싫어요
p_name:  다운동고분군
user_name:  강영송
place_rating:  4
place_comment:  옛것이 사라져가니 안타카울뿐이죠
p_name:  다운동고분군
user_name:  이재관 (부산모텔중개 꿈틀이)
place_rating:  5
place_comment:  
p_name:  다운동고분군
user_name:  홍성표
place_rating:  3
place_comment:  
p_name:  다운동고분군
user_name:  JS WOO
place_rating:  5
place_comment:  
p_name:  운흥사지
user_name:  강깁임
place_rating:  4
place_comment:  
65

C:\Users\82107\AppData\Local\Temp\ipykernel_32036\270387098.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./chromedriver", options=Options)


44
p_name:  생태습지
user_name:  Young Hee Kim
place_rating:  5
place_comment:  태풍이 지나간후라 그런지 모처럼 여름다운 날씨를 맛본것같습니다
전망대에 올라 등줄기를 시원하게 해준 한줄기의 바람.. 폭염으로 힘들었지만 습지체험을통해 건강한 수생식물들의 끝없이 나누어주는 자연의 고마움을 잘 알수 있었습니다
p_name:  생태습지
user_name:  방창순
place_rating:  5
place_comment:  잘 가꾸워 놓은
연밭이 아름다운
녹색 장관을 이루워
보기좋은곳
p_name:  생태습지
user_name:  김미미
place_rating:  3
place_comment:  연꽃이 물을 깨끗하게 정화하게 시킨다고합니다
꽃은 아직 피지 않았지만 8월이면 만개하겠죠
p_name:  생태습지
user_name:  옥작가
place_rating:  4
place_comment:  연밭으로 이루어진 생태공원이네요 ㆍ한번쯤 가볼만합니다
p_name:  생태습지
user_name:  최봄
place_rating:  5
place_comment:  연밭이 인상깊었고 자연의 보고인데 탐방하기 불편함없이 관리된것이 놀라웠다.
p_name:  생태습지
user_name:  슈쥬파파
place_rating:  5
place_comment:  아무나 갈수없는곳!
1년에 1달만 신청받는곳!
꽃이 손꼽을 정도라 아쉬움!
p_name:  생태습지
user_name:  이규철
place_rating:  5
place_comment:  보존가치가 충분합니다
p_name:  생태습지
user_name:  박도하
place_rating:  5
place_comment:  물을 아껴씁시다.
p_name:  생태습지
user_name:  SEO JU
place_rating:  5
place_comment:  
p_name:  생태습지
user_name:  토실깽이
place_rating:  5
place_comment:  Good


In [54]:
df.to_csv('kakaoGoogle.csv', encoding='utf-8-sig')